# Assignment 6.  
## Queries combining multiple tables

In this assigment, we will fetch data from a relational database server directly into Python variables. Using these new skills, we will complete the problems from the "Problems for you to Solve" in Chapters 8, 9, and 11 in *SQL Queries for Mere Mortals*.

### Increasing complexity of queries

When working with queries that address multiple tables, rely on the structure of the foreign keys and primary keys to guide the design of your queries. 

We will work with two basic patterns to solve most problems in the book:

1. Using a subquery in the `WHERE` clause
2. Inner natural join.

The database schemas in the book are designed so that foreign key columns share the same names as the primary key of their parent tables. This allows the use of natural joins for logically meaningful joins of tables related by the foreign key. 

First, let's connect to the database and activate the SQL magic for jupyter.

In [ ]:
import json
import pymysql 

pymysql.install_as_MySQLdb()

with open('cred.json') as f:
    creds = json.load(f)

connection_string = "mysql://{user}:{password}@{host}".format(**creds)

In [ ]:
%load_ext sql
%config SqlMagic.autocommit=True
%sql $connection_string

Let's solve the following query in the `shared_sales` database:
> Show all the customers who have bought a bicycle, i.e. products in category "Bike"

Review the schema diagram to follow how we build the query step-by-step.

In [ ]:
%%sql

USE shared_sales

In [ ]:
%%sql

show tables


In [ ]:
%%sql

-- Step 1:
-- category for Bike

SELECT * 
    FROM Categories 
    WHERE CategoryDescription = "Bikes"

In [ ]:
%%sql

-- Step 2
-- Products from that category
-- Datajoint: Product & (Categories & 'CategoryDescription="Bikes"')

SELECT * 
FROM Products
WHERE CategoryID in (SELECT CategoryID 
    FROM Categories 
    WHERE CategoryDescription = "Bikes")

In [ ]:
%%sql

-- Step 2
-- Products from that category, USING JOIN
-- DataJoint: Products * Categories & 'CategoryDescription = "Bikes"'

SELECT * 
    FROM Products NATURAL JOIN Categories
    WHERE CategoryDescription = "Bikes"


In [ ]:
%%sql

show tables

In [ ]:
%%sql

-- Step 3
-- Order details that contain such products
-- OrderDetails & (
--   Products * Categories & 'CategoryDescription = "Bikes"')

SELECT * 
FROM Order_Details
WHERE ProductNumber in (
    SELECT ProductNumber 
    FROM Products NATURAL JOIN Categories
    WHERE CategoryDescription = "Bikes")


In [ ]:
%%sql

-- Step 3
-- Order details that contain such products using JOINS
-- DataJoint: 
--   OrderDetails * Products * Categories & "CategoryDescription='Bikes'""

SELECT * 
FROM Order_Details NATURAL JOIN Products NATURAL JOIN Categories
WHERE CategoryDescription = "Bikes"

In [ ]:
%%sql

-- Step 4
-- Orders that contain such products 

SELECT * 
FROM Orders 
WHERE OrderNumber IN (
    SELECT OrderNumber
    FROM Order_Details NATURAL JOIN Products NATURAL JOIN Categories
    WHERE CategoryDescription = "Bikes")

Why can we not convert this into a pure join query?

In [ ]:
%%sql

-- Step 5
-- Customers on these orders

SELECT CustLastName 
FROM Customers
WHERE CustomerID IN (
    SELECT CustomerID    
    FROM Orders 
    WHERE OrderNumber IN (
        SELECT OrderNumber
        FROM Order_Details NATURAL JOIN Products NATURAL JOIN Categories
        WHERE CategoryDescription = "Bikes"))

In [ ]:
%%sql

-- Step 5
-- Customers on these orders


SELECT CustLastName 
FROM Customers
WHERE CustomerID in (
    SELECT CustomerID    
    FROM Orders 
        NATURAL JOIN Order_Details 
        NATURAL JOIN Products 
        NATURAL JOIN Categories 
    WHERE CategoryDescription = "Bikes")

In [ ]:
%%sql 

SELECT DISTINCT CustLastName 
FROM Customers 
        NATURAL JOIN Orders
        NATURAL JOIN Order_Details 
        NATURAL JOIN Products 
        NATURAL JOIN Categories 
    WHERE CategoryDescription = "Bikes"

In [ ]:
%%sql 
SELECT * FROM Order_Details JOIN Orders
ON Order_Details.OrderNumber = Orders.OrderNumber

In [ ]:
%%sql 
SELECT * FROM Order_Details JOIN Orders
USING (OrderNumber)

In [ ]:
%%sql 
SELECT * FROM Order_Details NATURAL JOIN Orders

These two types of queries: equi-joins (including natural joins) and subqueries in the `WHERE` clauses can be used to solve the majority of problems in the assignment.

# Different types of equijoins

In [ ]:
%%sql
use shared_sales;
show tables;

In [ ]:
%%sql

SELECT Customers.CustomerID, Orders.CustomerID FROM Customers, Orders;


In [ ]:
%%sql

SELECT Customers.CustomerID, Orders.CustomerID FROM Customers JOIN Orders;

In [ ]:
%%sql

SELECT * FROM Customers JOIN Orders WHERE Customers.CustomerID=Orders.CustomerID

In [ ]:
%%sql

SELECT * FROM Customers JOIN Orders ON Customers.CustomerID = Orders.CustomerID 

In [ ]:
%%sql

SELECT * FROM Customers JOIN Orders USING (CustomerID) 

In [ ]:
%%sql

SELECT * FROM Customers NATURAL JOIN Orders 

In [ ]:
%%sql

SELECT count(*) FROM Customers NATURAL JOIN Employees NATURAL JOIN Orders;

### Example

Display recipe types that do have any recipes

DataJoint: RecipeType - Recipe

In [ ]:
%%sql
use shared_recipes;
show tables;

In [ ]:
%%sql

SELECT RecipeClassDescription 
   FROM Recipe_Classes 
   WHERE RecipeClassID not in (
       SELECT RecipeClassID 
       FROM Recipes)

### Example 
Show orders placed on Oct 3, 2017, include customer name.

In [ ]:
%%sql
use shared_sales;

SELECT Orders.*, CustFirstName, CustLastName 
   FROM Customers NATURAL JOIN Orders
    WHERE OrderDate="2017-10-03"

In [ ]:
#
Give me the most expensive product

In [ ]:
%%sql

SELECT max(RetailPrice) FROM Products

In [ ]:
%%sql

SELECT * FROM Products WHERE RetailPrice = (SELECT max(RetailPrice) FROM Products)

#  Homework

**Problem 1**: Sales: Show all orders with their details, including the customer name 

(944)

**Problem 2**: Sales: Show all employee/customer pairs who have ever been on the same order

(211)

**Problem 3**: Sales: Show all order items with the total amount billed for them. Include product names.

(3973)

**Problem 4**: Sales. Show customers and employees who have the same last name.

(16)

**Problem 6**: Entertainment. Show engagements sorted by date, including agent name.

(111)

**Problem 7**: Entertainment. List the names of all entertainer/agent pairings who live in the same postal code.

(10)

**Problem 8**: School. List current student class enrollments, including students' names and class names. Sort by student last name.

(50)

In [ ]:
%%sql

SELECT StudFirstName, StudLastName, SubjectName  
    FROM Students 
        NATURAL JOIN Student_Schedules 
        NATURAL JOIN Classes 
        NATURAL JOIN Subjects 
    WHERE ClassStatus=1
    ORDER BY StudLastName

**Problem 9**: School. Show students with a grade 85 or better in ART and 85 or better in a CIS course.

(1)

In [ ]:
%%sql
use shared_school;
show tables;

In [ ]:
%%sql
SELECT StudFirstName, StudLastName FROM Students 
WHERE StudentID in (
    SELECT StudentID 
        FROM Student_Schedules NATURAL JOIN Classes NATURAL JOIN Subjects 
        WHERE Grade>=85 and CategoryID="ART") 
    AND StudentID in (
    SELECT StudentID 
        FROM Student_Schedules NATURAL JOIN Classes NATURAL JOIN Subjects 
        WHERE Grade>=85 and CategoryID="CIS")